In [1]:
library("GenomicRanges")
#remotes::install_github("daewoooo/primatR", force=TRUE)
library("primatR")
library(dplyr)

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The

In [27]:
SVs_info <- read.table(
    "/public/home/zpliu/Pan-genome/SV_parallele_V2/superPan/Dt_genotype_V2/Dt_SV_merge_info.txt",
    sep = "\t",
    comment.char = "",
    header = T
)

In [28]:
SVs_info=SVs_info %>%
    select(c("X.Chr", "start", "end")) %>%
    mutate(
        Site = ceiling(
            (start + end) / 2
        )
    )
colnames(SVs_info)=c("Chr",'Start',"End","Site")

In [29]:
gr <- GRanges(
    seqnames = Rle(SVs_info %>% pull(Chr)),
    ranges = IRanges(
        start = SVs_info %>% pull(Site),
        end = SVs_info %>% pull(Site) + 1
    )
)

In [30]:
genomeLengths=read.table(
    "/public/home/zpliu/Pan-genome/Cactus-Pan/genomeSequence/HC04/HC04-softMasked.fa.fai",
    header=F,sep="\t"
)
#* 添加染色体长度信息
At_length=genomeLengths[grep("^HC04_D",genomeLengths$V1),] %>% pull(V2)
seqlengths(gr)=At_length

In [24]:
#A2的染色体信息
genomeLengths=read.table(
    "/public/home/zpliu/Pan-genome/Cactus-Pan/genomeSequence/J85/J85-softMasked.fa.fai",
    header=F,sep="\t"
)
#* 添加染色体长度信息
At_length=genomeLengths[grep("^J85",genomeLengths$V1),] %>% pull(V2)
seqlengths(gr)=At_length

In [31]:
#! hotspots分析
data=hotspotter(
    gr,bw = 200000,num.trial = 1000,
    pval = 0.05
)

In [32]:
write.table(
    as.data.frame(data),
    file = "Dt_hotpots.txt",
    row.names = F,
    col.names = T,
    sep = "\t",
    quote = F
)